In [1]:
%load_ext autoreload

In [23]:
%autoreload 2
import hydra
import sys
import torch

from RegularizingEmbeddings.data.data_generation import make_trajectories, postprocess_data, create_dataloaders
from RegularizingEmbeddings.data.data_utils import generate_train_and_test_sets

In [20]:
# load the config
with hydra.initialize(version_base="1.3", config_path="../conf"):
    cfg = hydra.compose(config_name="config")


In [24]:
torch.random.manual_seed(cfg.data.flow.random_state)
eq, sol, dt = make_trajectories(cfg)

# Postprocess data
values = postprocess_data(cfg, sol)

# Create train and test sets
train_dataset, val_dataset, test_dataset, trajs = generate_train_and_test_sets(values, **cfg.data.train_test_params)